# Fast scan
The goal for this algo is to scan efficiently a grid to find a winner.
Let's make sure we do not forget a corner case.

In [1]:
import numpy as np
from TicTacToe import *

In [2]:
grid = np.zeros((20, 20))
pts = (1, 6, -10, -16)
p1c1, p1c2, p2c1, p2c2 = pts
max_l = 5
ww = 2*max_l-1  # Window width

subgrid = np.zeros((9, 9), dtype=np.int8)
p1c1_pts = {
    (0, 0), (1, 1), (2, 2), (3, 3),
    (3, 1),
    (4, 0), (4, 1), (4, 2), (4, 3),
    (5, 1), (6, 1),
    (2, 4),
    (4, 5)
}
for i, j in p1c1_pts:
    subgrid[i, j] = p1c1
    
p1c2_pts = {
    
    (2, 7), (3, 7), (4, 7), (5, 7), (6, 7)
}
for i, j in p1c2_pts:
    subgrid[i, j] = p1c2


p2c1_pts = {
    (5, 2), (1, 4), (2, 3),
    (3, 4), (3, 5), (3, 6),
    (2, 6), (3, 6), (4, 6)
}
for i, j in p2c1_pts:
    subgrid[i, j] = p2c1

p2c2_pts = {
    (6, 2), (6, 3), (6, 4), (6, 5), (6, 6)
}
for i, j in p2c2_pts:
    subgrid[i, j] = p2c2



In [3]:
# If a p1c1 is added in the center, how many points are scored? -> Should be 1
p1win = {max_l*p1c1, (max_l-1)*p1c1 + p1c2}
p2win = {max_l*p2c1, (max_l-1)*p2c1 + p2c2}

In [4]:
subgrid

array([[  1,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   1,   0,   0, -10,   0,   0,   0,   0],
       [  0,   0,   1, -10,   1,   0, -10,   6,   0],
       [  0,   1,   0,   1, -10, -10, -10,   6,   0],
       [  1,   1,   1,   1,   0,   1, -10,   6,   0],
       [  0,   1, -10,   0,   0,   0,   0,   6,   0],
       [  0,   1, -16, -16, -16, -16, -16,   6,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int8)

In [5]:
"""# [DEBUG] Tic Tac Toe version
max_l = 3
ww = 3
p1win = {max_l*p1c1, (max_l-1)*p1c1 + p1c2}
p2win = {max_l*p2c1, (max_l-1)*p2c1 + p2c2}
subgrid = np.array([[p1c1, 0, 0], [p1c1, 0, 0], [p1c1, 0, 0]], dtype=np.int8)
subgrid
"""

'# [DEBUG] Tic Tac Toe version\nmax_l = 3\nww = 3\np1win = {max_l*p1c1, (max_l-1)*p1c1 + p1c2}\np2win = {max_l*p2c1, (max_l-1)*p2c1 + p2c2}\nsubgrid = np.array([[p1c1, 0, 0], [p1c1, 0, 0], [p1c1, 0, 0]], dtype=np.int8)\nsubgrid\n'

In [6]:
p1win

{5, 10}

In [7]:
#i, j = 8, 8
rmax, cmax = subgrid.shape
#assert 0 <= i < ww and 0 <= j < ww
for i in range(ww):
    for j in range(ww):
        temp_ = subgrid[i, j]
        subgrid[i, j] = 100  # Place token
        k, rl, cl = boardCheck.extractSubgrid(subgrid, i, j)
        assert k[rl, cl] == 100
        subgrid[i, j] = temp_  # Revert changes

In [8]:
# Reset scores
p1 = {'name': 'p1', 'win': p1win, 'score': 0, 'token_first': p1c1, 'token_assigned': p1c2}
p2 = {'name': 'p2', 'win': p2win, 'score': 0, 'token_first': p2c1, 'token_assigned': p2c2}

In [9]:
# [TEST] Go through the subgrid, extract relevant array and count points
def test_countSubgrid(subgrid):
    for i in range(ww):
        for j in range(ww):
            #i, j = max_l-1, max_l-1  # Local coordinates of the last token placed
            temp_ = subgrid[i, j]
            subgrid[i, j] = p1['token_first']  # Place token
            sg, r, c = boardCheck.extractSubgrid(subgrid, i, j)  # Get subgrid
            boardCheck.check_row(sg, r, p1)
            boardCheck.check_col(sg, c, p1)
            boardCheck.check_hills(sg, r, c, p1)
            boardCheck.check_dales(sg, r, c, p1)
            subgrid[i, j] = temp_
%timeit test_countSubgrid(subgrid)

1.72 ms ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
print(p1['score'], p2['score'])

3 0


In [11]:
subgrid

array([[  6,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   6,   0,   0, -10,   0,   0,   0,   0],
       [  0,   6,   6, -10,   6,   0, -10,   6,   0],
       [  0,   6,   0,   6, -10, -10, -10,   6,   0],
       [  6,   6,   6,   6,   0,   1, -10,   6,   0],
       [  0,   6, -10,   0,   0,   0,   0,   6,   0],
       [  0,   6, -16, -16, -16, -16, -16,   6,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int8)